In [56]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk  # For displaying images
import psycopg2

class LibraryApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Library System")
        self.geometry("400x700")
        self.configure(bg='#002D72')

        self.current_frame = None
        self.switch_frame(ConfirmFrame)

    def switch_frame(self, frame_class, *args, **kwargs):
        new_frame = frame_class(self, *args, **kwargs)
        if self.current_frame is not None:
            self.current_frame.destroy()
        self.current_frame = new_frame
        self.current_frame.pack(fill=tk.BOTH, expand=True)

class ConfirmFrame(tk.Frame):
    def __init__(self, master):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.create_widgets()

    def create_widgets(self):
        title_label = tk.Label(self, text="Bạn đăng nhập với tư cách nào?", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        reader_button = tk.Button(self, text="Độc giả", command=self.login_as_reader, width=20, height=2)
        reader_button.pack(pady=10)

        staff_button = tk.Button(self, text="Nhân viên", command=self.login_as_staff, width=20, height=2)
        staff_button.pack(pady=10)

    def login_as_reader(self):
        self.master.switch_frame(LoginFrame, 'reader')

    def login_as_staff(self):
        self.master.switch_frame(LoginFrame, 'staff')

class LoginFrame(tk.Frame):
    def __init__(self, master, user_type):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.user_type = user_type
        self.create_widgets()

    def create_widgets(self):
        title_label = tk.Label(self, text="Đăng nhập", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        username_label = tk.Label(self, text="Tên đăng nhập", fg="white", bg='#002D72')
        username_label.pack(pady=(5, 5))
        self.username_entry = tk.Entry(self, width=30)
        self.username_entry.pack(pady=(5, 10))

        password_label = tk.Label(self, text="Mật khẩu", fg="white", bg='#002D72')
        password_label.pack(pady=(5, 5))
        self.password_entry = tk.Entry(self, show='*', width=30)
        self.password_entry.pack(pady=(5, 10))

        login_button = tk.Button(self, text="Đăng nhập", command=self.login, width=20)
        login_button.pack(pady=20)

        register_label = tk.Label(self, text="Bạn chưa có tài khoản, hãy đăng ký", fg="white", bg='#002D72', cursor="hand2")
        register_label.pack(pady=(20, 10))
        register_label.bind("<Button-1>", lambda e: self.master.switch_frame(RegisterFrame))

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        # Database connection and query
        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port="5432"
            )
            cursor = conn.cursor()
            cursor.execute(f"SELECT * FROM ACCOUNT WHERE login_name=%s AND password=%s", (username, password))
            account = cursor.fetchone()
            cursor.close()
            conn.close()

            if account:
                messagebox.showinfo("Login Success", "Đăng nhập thành công!")
                if self.user_type == 'reader':
                    self.master.switch_frame(ReaderFrame, account[0])  # Pass account_id to ReaderFrame
                else:
                    self.master.switch_frame(StaffFrame, account[0])
            else:
                messagebox.showwarning("Login Failed", "Tên đăng nhập hoặc mật khẩu chưa đúng")
        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")

class RegisterFrame(tk.Frame):
    def __init__(self, master):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.create_widgets()

    def create_widgets(self):
        title_label = tk.Label(self, text="Đăng ký tài khoản", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        self.fields = {
            "Họ và tên": tk.Entry(self, width=30),
            "Giới tính": tk.Entry(self, width=30),
            "Ngày tháng năm sinh": tk.Entry(self, width=30),
            "Số điện thoại": tk.Entry(self, width=30),
            "Địa chỉ email": tk.Entry(self, width=30),
            "Tên đăng nhập": tk.Entry(self, width=30),
            "Mật khẩu": tk.Entry(self, width=30, show='*')
        }

        for label_text, entry in self.fields.items():
            label = tk.Label(self, text=label_text, fg="white", bg='#002D72')
            label.pack(pady=(5, 5))
            entry.pack(pady=(5, 10))

        register_button = tk.Button(self, text="Đăng ký", command=self.register, width=20)
        register_button.pack(pady=20)

    def register(self):
        data = {label_text: entry.get() for label_text, entry in self.fields.items()}
        # Insert into database
        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port = "5432"
            )
            cursor = conn.cursor()
            cursor.execute(
                "INSERT INTO ACCOUNT (login_name, password, register_date, role) VALUES (%s, %s, CURRENT_DATE, %s) RETURNING account_id",
                (data["Tên đăng nhập"], data["Mật khẩu"], False)
            )
            account_id = cursor.fetchone()[0]
            cursor.execute(
                "INSERT INTO READER (reader_id, full_name, gender, dob, email, phone_number, account_id) VALUES (DEFAULT, %s, %s, %s, %s, %s, %s)",
                (data["Họ và tên"], data["Giới tính"], data["Ngày tháng năm sinh"], data["Địa chỉ email"], data["Số điện thoại"], account_id)
            )
            conn.commit()
            cursor.close()
            conn.close()
            messagebox.showinfo("Register Success", "Đăng ký thành công!")
            self.master.switch_frame(LoginFrame, 'reader')
        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")

class ReaderFrame(tk.Frame):
    def __init__(self, master, account_id):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.account_id = account_id
        self.create_widgets()

    def create_widgets(self):
        # Retrieve the user's name from the database
        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port="5432"
            )
            cursor = conn.cursor()
            cursor.execute("SELECT full_name FROM READER WHERE account_id=%s", (self.account_id,))
            user_name = cursor.fetchone()[0]
            cursor.close()
            conn.close()
        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")
            user_name = "User"

        title_label = tk.Label(self, text=f"Welcome, {user_name}!", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        question_label = tk.Label(self, text="Bạn muốn làm gì?", fg="white", bg='#002D72', font=("Helvetica", 14))
        question_label.pack(pady=(10, 20))

        book_search_button = tk.Button(self, text="Tra cứu các đầu sách hiện có", width=30, command=self.search_books)
        book_search_button.pack(pady=10)

        borrow_history_button = tk.Button(self, text="Tra cứu lịch sử mượn/trả sách của bạn", width=30, command=self.view_borrow_history)
        borrow_history_button.pack(pady=10)

        room_search_button = tk.Button(self, text="Tra cứu về các phòng của thư viện", width=30, command=self.search_rooms)
        room_search_button.pack(pady=10)

        exit_app_button = tk.Button(self, text = "Thoát khỏi ứng dụng", width = 30, command = self.exit_app)
        exit_app_button.pack(pady=10)

    def search_books(self):
        self.master.switch_frame(BookSearchFrame, self.account_id)

    def view_borrow_history(self):
        self.master.switch_frame(BorrowReturnSearchFrame, self.account_id)

    def search_rooms(self):
        self.master.switch_frame(RoomSearchFrame, self.account_id)
    
    def exit_app(self):
        self.master.switch_frame(ConfirmFrame)

class BookSearchFrame(tk.Frame):
    def __init__(self, master, account_id):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.account_id = account_id
        self.create_widgets()

    def create_widgets(self):
        title_label = tk.Label(self, text="Tìm kiếm sách", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        book_name_label = tk.Label(self, text="Tên sách", fg="white", bg='#002D72')
        book_name_label.pack(pady=(5, 5))
        self.book_name_entry = tk.Entry(self, width=30)
        self.book_name_entry.pack(pady=(5, 10))

        search_button = tk.Button(self, text="Tìm kiếm", command=self.search_book, width=20)
        search_button.pack(pady=20)

        self.result_frame = tk.Frame(self, bg='#002D72')
        self.result_frame.pack(pady=(10, 10))

    def search_book(self):
        book_name = self.book_name_entry.get()

        for widget in self.result_frame.winfo_children():
            widget.destroy()

        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port="5432"
            )
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM Search_book WHERE title=%s", (book_name,))
            book = cursor.fetchone()
            cursor.close()
            conn.close()

            if book:
                book_info = f"Tên sách: {book[0]}\nTác giả: {book[1]}\nNăm xuất bản: {book[2]}\nSố trang: {book[3]}\nNhà xuất bản: {book[4]}\nThể loại: {book[5]}\nSố lượng hiện có: {book[6]}\nPhòng mượn: {book[7]}"
                book_label = tk.Label(self.result_frame, text=book_info, fg="white", bg='#002D72', anchor="w", justify="left")
                book_label.pack(pady=(10, 10), fill = tk.X, padx = 10)

            else:
                not_found_label = tk.Label(self.result_frame, text="Thư viện không có đầu sách bạn mong muốn", fg="red", bg='#002D72')
                not_found_label.pack(pady=(10, 10))

        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")

        back_button = tk.Button(self.result_frame, text="Quay lại", command=self.back_to_reader, width=20)
        back_button.pack(pady=20)

    def back_to_reader(self):
        self.master.switch_frame(ReaderFrame, self.account_id)

class BorrowReturnSearchFrame(tk.Frame):
    def __init__(self, master, account_id):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.account_id = account_id
        self.create_widgets()

    def create_widgets(self):
        title_label = tk.Label(self, text="Lịch sử mượn/trả sách", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        search_button = tk.Button(self, text="Tìm kiếm", command=self.load_borrow_return_history, width=20)
        search_button.pack(pady=20)

        self.result_frame = tk.Frame(self, bg='#002D72')
        self.result_frame.pack(pady=(10, 10))

    def load_borrow_return_history(self):
        
        for widget in self.result_frame.winfo_children():
            widget.destroy()

        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port="5432"
            )
            cursor = conn.cursor()
            cursor.execute(
                "SELECT * FROM Search_borrow_return WHERE account_id = %s", (self.account_id,)
            )
            history = cursor.fetchall()
            cursor.close()
            conn.close()

            for record in history:
                if record:
                    history_info = f"Tên sách: {record[2]}\nNgày mượn: {record[3]}\nSố lượng: {record[4]}\nNgày đến hạn: {record[5]}\nNgày trả: {record[6] if record[6] else 'Chưa trả'}"
                    history_label = tk.Label(self.result_frame, text=history_info, fg="white", bg='#002D72', anchor="w", justify="left")
                    history_label.pack(pady=(10, 10), fill = tk.X, padx = 10)

                else:
                    not_found_label = tk.Label(self.result_frame, text="Bạn chưa mượn hay trả đầu sách nào", fg="red", bg='#002D72')
                    not_found_label.pack(pady=(10, 10))

        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")

        back_button = tk.Button(self.result_frame, text="Quay lại", command=self.back_to_reader, width=20)
        back_button.pack(pady=20)

    def back_to_reader(self):
        self.master.switch_frame(ReaderFrame, self.account_id)

class RoomSearchFrame(tk.Frame):
    def __init__(self, master, account_id):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.account_id = account_id
        self.create_widgets()

    def create_widgets(self):
        title_label = tk.Label(self, text="Tìm kiếm phòng", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        room_name_label = tk.Label(self, text="Tên phòng", fg="white", bg='#002D72')
        room_name_label.pack(pady=(5, 5))
        self.room_name_entry = tk.Entry(self, width=30)
        self.room_name_entry.pack(pady=(5, 10))

        search_button = tk.Button(self, text="Tìm kiếm", command=self.search_room, width=20)
        search_button.pack(pady=20)

        self.result_frame = tk.Frame(self, bg='#002D72')
        self.result_frame.pack(pady=(10, 10))
   
    def search_room(self):
        room_name = self.room_name_entry.get()

        for widget in self.result_frame.winfo_children():
            widget.destroy()

        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port="5432"
            )
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM Search_room WHERE room_name = %s", (room_name,))
            room = cursor.fetchone()
            cursor.close()
            conn.close()

            if room:
                room_info = f"Tên phòng: {room[0]}\nChức năng: {room[1]}\nTrưởng phòng: {room[2]}\nLiên hệ: {room[3]}"
                room_label = tk.Label(self.result_frame, text=room_info, fg="white", bg='#002D72', anchor="w", justify="left")
                room_label.pack(pady=(10, 10), fill = tk.X, padx = 10)

            else:
                not_found_label = tk.Label(self.result_frame, text="Thư viện không có phòng này", fg="red", bg='#002D72')
                not_found_label.pack(pady=(10, 10))
        
        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")

        back_button = tk.Button(self.result_frame, text="Quay lại", command=self.back_to_reader, width=20)
        back_button.pack(pady=20)

    def back_to_reader(self):
        self.master.switch_frame(ReaderFrame, self.account_id)

class StaffFrame(tk.Frame):
    def __init__(self, master, account_id):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.account_id = account_id
        self.create_widgets()

    def create_widgets(self):
        # Retrieve the user's name from the database
        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port="5432"
            )
            cursor = conn.cursor()
            cursor.execute("SELECT full_name FROM STAFF WHERE account_id=%s", (self.account_id,))
            user_name = cursor.fetchone()[0]
            cursor.close()
            conn.close()
        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")
            user_name = "User"

        title_label = tk.Label(self, text=f"Welcome, {user_name}!", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        question_label = tk.Label(self, text="Bạn muốn làm gì?", fg="white", bg='#002D72', font=("Helvetica", 14))
        question_label.pack(pady=(10, 20))

        book_search_button = tk.Button(self, text="Nhập sách về thư viện", width=30, command=self.import_books)
        book_search_button.pack(pady=10)

        borrow_history_button = tk.Button(self, text="Xác nhận bạn đọc mượn sách", width=30, command=self.reader_borrow)
        borrow_history_button.pack(pady=10)

        room_search_button = tk.Button(self, text="Xác nhận bạn đọc trả sách", width=30, command=self.reader_return)
        room_search_button.pack(pady=10)

        exit_app_button = tk.Button(self, text = "Thoát khỏi ứng dụng", width = 30, command = self.exit_app)
        exit_app_button.pack(pady=10)

    def import_books(self):
        self.master.switch_frame(ImportBookFrame, self.account_id)

    def reader_borrow(self):
        self.master.switch_frame(ReaderBorrowFrame, self.account_id)

    def reader_return(self):
        self.master.switch_frame(ReaderReturnFrame, self.account_id)

    def exit_app(self):
        self.master.switch_frame(ConfirmFrame)

class ImportBookFrame(tk.Frame):
    def __init__(self, master, account_id):
        super().__init__(master, bg='#002D72')
        self.master = master
        self.account_id = account_id
        self.create_widgets()

    def create_widgets(self):
        title_label = tk.Label(self, text="Xác nhận nhập sách", fg="white", bg='#002D72', font=("Helvetica", 16))
        title_label.pack(pady=(50, 20))

        self.fields = {
            "Mã sách nhập về": tk.Entry(self, width=30),
            "Số lượng nhập": tk.Entry(self, width=30),
            "Ngày nhập": tk.Entry(self, width=30),
        }

        for label_text, entry in self.fields.items():
            label = tk.Label(self, text=label_text, fg="white", bg='#002D72')
            label.pack(pady=(5, 5))
            entry.pack(pady=(5, 10))

        register_button = tk.Button(self, text="Xác nhận", command=self.confirm, width=20)
        register_button.pack(pady=20)

        back_button = tk.Button(self, text="Quay lại", command=self.go_back, width=20)
        back_button.pack(pady=20)

    def confirm(self):
        data = {label_text: entry.get() for label_text, entry in self.fields.items()}
        # Insert into database
        try:
            conn = psycopg2.connect(
                dbname="library_db", user="postgres", password="hv19062004", host="localhost", port = "5432"
            )
            cursor = conn.cursor()
            cursor.execute(
                "SELECT staff_id FROM staff WHERE account_id =%s", (self.account_id,)
            )
            staff_id_result = cursor.fetchone()

            cursor.execute(
                "INSERT INTO IMPORT (import_date, number_of_copies, confirmer_id, isbn) VALUES (%s, %s, %s, %s)",
                (data["Ngày nhập"], data["Số lượng nhập"], staff_id_result, data["Mã sách nhập về"])
            )

            conn.commit()
            cursor.close()
            conn.close()
            messagebox.showinfo("Register Success", "Xác nhận thành công!")
        
        except psycopg2.Error as e:
            messagebox.showerror("Database Error", f"Error: {e}")
    
    def go_back(self):
        self.master.switch_frame(StaffFrame, self.account_id)

if __name__ == "__main__":
    app = LibraryApp()
    app.mainloop()